In [0]:
from urllib.request import urlopen 
from bs4 import BeautifulSoup 
import pandas as pd

##Scrape 
Get per game and advanced stats for each year and join on the columns (Player, Pos, Age, Tm, G) as these are common 

In [0]:
def get_stats(year, statstype):
  url = "https://www.basketball-reference.com/leagues/NBA_{}_{}.html".format(year,statstype)
  html = urlopen(url)
  soup = BeautifulSoup(html)
  #soup.findAll('tr', limit=2)

  headers = [th.getText() for th in soup.findAll('tr',limit=2)[0].findAll('th')]
  headers = headers[1:]

  rows = soup.findAll('tr')[1:]
  player_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]
  stats = pd.DataFrame(player_stats, columns = headers)
  return stats.drop_duplicates().dropna(how='all')


len(get_stats(2019,'per_game'))

  



708

In [0]:
def get_joint_stats(year):
  per_game = get_stats(year,'per_game')
  advanced = get_stats(year,'advanced')
  joint = pd.merge(per_game,advanced, on=['Player', 'Pos', 'Age', 'Tm', 'G'], how='inner', suffixes=('_per_game','_advanced'))
  return joint

len(get_joint_stats(2019))

708

In [0]:
#All stats for all years
initial = get_joint_stats(1983)
for year in range (1984,2020):
  joint = pd.merge(initial, get_joint_stats(year), on='Player', how='outer', suffixes=('','_year'))